BERT_Model Pretraining

1、Set a toy data to train.As you know,here i just list an example,if you want to train formally,you can prepare superior dataset!

In [1]:
toy_data = [
    {"text": "I love machine learning."},
    {"text": "It is a fascinating field."},
    {"text": "BERT is a type of transformer."},
    {"text": "Transformers are powerful models."},
]

2、Then define a dataloader

In [2]:
from transformers import BertTokenizer
from torch.utils.data import DataLoader
from Datapreprocess import PretrainDataset, collate_fn



tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

dataset = PretrainDataset(toy_data,tokenizer)

from torch.utils.data import DataLoader

dataloader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=lambda batch: collate_fn(batch, pad_token_id=tokenizer.pad_token_id)
)


f:\toolbox\python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
f:\toolbox\python\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


3、Now need to load model

In [8]:
from modules.bert import BertPretrainingModel
import torch

# Initialize the model
model = BertPretrainingModel(
    vocab_size=tokenizer.vocab_size,
    type_vocab_size = 2,
    hidden_size = 256,
    max_len = 64,
    num_layers = 2,
    intermidiate_size=512,
    num_attention_head = 8,
    dropout = 0.1,
    padding_idx=tokenizer.pad_token_id
)

optimizer = torch.optim.Adam(model.parameters(),lr = 1e-4)
model.train()

for epoch in range(3):
    for batch_idx, batch in enumerate(dataloader):
        optimizer.zero_grad()
        output = model(
            batch.input_ids,
            batch.token_type_ids,
            labels=batch.labels,
            next_sentence_label=batch.is_next
        )
        loss = output[0]
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

        # 只打印前2个batch的mask和预测
        if epoch == 0 and batch_idx < 2:
            mask_pos = (batch.labels != -1)
            # 1. 还原原始token序列
            input_tokens = [tokenizer.convert_ids_to_tokens(ids) for ids in batch.input_ids]
            # 2. 还原被mask的真实token
            masked_ids = batch.labels[mask_pos]
            masked_tokens = tokenizer.convert_ids_to_tokens(masked_ids.tolist())
            # 3. 还原模型预测token
            pred_ids = output[2].argmax(-1)[mask_pos]
            pred_tokens = tokenizer.convert_ids_to_tokens(pred_ids.tolist())
            # 4. 打印每个样本
            for i in range(batch.input_ids.size(0)):
                print(f"\n样本{i+1}原句token: {input_tokens[i]}")
                # 找到该样本的mask位置
                sample_mask_pos = mask_pos[i]
                sample_masked_tokens = [token for token, m in zip(masked_tokens, sample_mask_pos.tolist()) if m]
                sample_pred_tokens = [token for token, m in zip(pred_tokens, sample_mask_pos.tolist()) if m]
                print("被mask的真实token:", sample_masked_tokens)
                print("模型预测token:", sample_pred_tokens)
            print("NSP真实标签:", batch.is_next)
            print("NSP预测:", output[3].argmax(-1))
        

Epoch 1, Loss: 0.7492763996124268

样本1原句token: ['[CLS]', 'it', 'is', 'a', 'fascinating', 'field', '.', '[SEP]', 'bert', 'is', 'a', 'type', 'of', 'transform', '##er', '.', '[SEP]']
被mask的真实token: ['[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
模型预测token: ['statutes', 'nutrition', '[unused355]', 'logged', 'у', 'sums', 'postseason', 'associate', 'dreams', 'waved', 'sort', 'staircase', 'obtain', 'amnesty', 'verve', 'kelly', 'bedrock']

样本2原句token: ['[CLS]', 'bert', 'is', 'a', 'type', 'of', 'transform', '##er', '.', '[SEP]', 'transformers', 'are', 'powerful', 'models', '.', '[SEP]', '[PAD]']
被mask的真实token: ['[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
模型预测token: ['statutes', 'nutrition', '[unused355]', 'logged', 'у', 'sums', 'postseason', 'associate', 'dreams', 'waved', 'sort', 's